<img style="float: left;" src="./images/logo.png" width="400"/>

this code is from the official GFED website

In [7]:
months       = '01','02','03','04','05','06','07','08','09','10','11','12'
sources      = 'SAVA','BORF','TEMF','DEFO','PEAT','AGRI'

in this example we will calculate annual CO emissions for the 14 GFED basisregions over 1997-2014. 

Please adjust the code to calculate emissions for your own specie, region, and time period of interest. 

Please first download the GFED4.1s files (available on the dropbox folder for simplicity) and the GFED4_Emission_Factors_Summary.csv to your computer and adjust the directory where you placed them below

In [10]:
directory    = './GFED4.1s/data/'

## Read in emission factors

In [14]:
species = [] # names of the different gas and aerosol species
EFs     = np.zeros((41, 6)) # 41 species, 6 sources

data_EF = pd.read_csv(directory+'/GFED4_Emission_Factors_Summary.csv', header=10)
EF_CO = np.array(data_EF.iloc[3])[1:]


# we are interested in CO for this example (4th row):

start_year = 1997
end_year   = 2014

./GFED4.1s/data//GFED4_Emission_Factors_Summary.csv


## Make table with summed DM emissions for each region, year, and source

In [15]:

CO_table = np.zeros((15, end_year - start_year + 1)) # region, year

for year in range(start_year, end_year+1):
    string = directory+'/GFED4.1s_'+str(year)+'.hdf5'
    f = h5py.File(string, 'r')
    
    
    if year == start_year: # these are time invariable    
        basis_regions = f['/ancill/basis_regions'][:]
        grid_area     = f['/ancill/grid_cell_area'][:]
    
    
    CO_emissions = np.zeros((720, 1440))
    for month in range(12):
        # read in DM emissions
        string = '/emissions/'+months[month]+'/DM'
        DM_emissions = f[string][:]
        for source in range(6):
            # read in the fractional contribution of each source
            string = '/emissions/'+months[month]+'/partitioning/DM_'+sources[source]
            contribution = f[string][:]
            # calculate CO emissions as the product of DM emissions (kg DM per 
            # m2 per month), the fraction the specific source contributes to 
            # this (unitless), and the emission factor (g CO per kg DM burned)
            CO_emissions += DM_emissions * contribution * float(EF_CO[source])
    
    
    # fill table with total values for the globe (row 15) or basisregion (1-14)
    for region in range(15):
        if region == 14:
            mask = np.ones((720, 1440))
        else:
            mask = basis_regions == (region + 1)            
    
        CO_table[region, year-start_year] = np.sum(grid_area * mask * CO_emissions)
        
    print(year)
        
# convert to Tg CO 
CO_table = CO_table / 1E12
print(CO_table)

# please compare this to http://www.falw.vu/~gwerf/GFED/GFED4/tables/GFED4.1s_CO.txt

1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
[[6.71255991e+00 2.41162946e+01 1.00481924e+01 5.52384573e+00
  3.03693173e+00 2.32152092e+01 1.45367498e+01 2.70896856e+01
  1.86817257e+01 1.33120174e+01 1.02852082e+01 8.25607148e+00
  1.07677123e+01 1.93295823e+01 1.31535623e+01 1.36819172e+01
  2.71704451e+01 3.11602986e+01]
 [1.82754076e+00 2.19665292e+00 3.45271535e+00 3.42334954e+00
  1.89419318e+00 2.92623976e+00 2.77244789e+00 2.16927161e+00
  2.45677272e+00 3.24629060e+00 4.73834953e+00 3.53797177e+00
  2.42795661e+00 2.85934147e+00 3.25030233e+00 4.14140232e+00
  2.70653410e+00 3.03882031e+00]
 [3.53285801e+00 3.00030651e+01 4.04475727e+00 9.25639218e+00
  2.61308432e+00 3.50916123e+00 1.08710726e+01 2.59440922e+00
  8.54783393e+00 4.66596291e+00 5.27897718e+00 4.54854511e+00
  5.32021173e+00 3.22944476e+00 7.36468347e+00 2.70172197e+00
  6.52527138e+00 2.41144759e+00]
 [5.34896412e+00 7.50056439e+00 3.99634135e+00 5.21914089e+00
  5.6